<a href="https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Piper](https://github.com/rhasspy/piper) model exporter

Notebook created by [rmcpantoja](http://github.com/rmcpantoja)

In [ ]:
#@title Install software
#@markdown **Note: Please restart the runtime environment if prompted, and then continue to the next cell.**
!git clone https://github.com/rhasspy/piper
%cd /content/piper/src/python
!pip install --upgrade pip
!pip install cython>=0.29.0 espeak-phonemizer>=1.1.0 librosa>=0.9.2 numpy>=1.19.0 pytorch-lightning~=1.7.0 torch~=1.11.0
!pip install onnx onnxruntime-gpu
!bash build_monotonic_align.sh
!apt-get install espeak-ng
!pip install torchtext==0.12.0

In [ ]:
#@title Voice package generation section
%cd /content/piper/src/python
import os
import ipywidgets as widgets
from IPython.display import display
import json
from google.colab import output
#@markdown #### Download:
#@markdown **Google Drive model ID:**
model_id = "1LQ8CY8stAhk6aK2HbDxNOvff8vT0q7Jr+" #@param {type:"string"}
#@markdown **Config.json Google Drive ID**
config_id = "1-7qJ-Aw8OmqKgxxG8o_onU_Rwn0xfIOj" #@param {type:"string"}
#@markdown ---

#@markdown #### Creation process:
language = "es" #@param ["en-us", "es", "fr"]
voice_name = "Dabe" #@param {type:"string"}
voice_name = voice_name.lower()
quality = "medium" #@param ["high", "low", "medium", "x-low"]
export_voice_name = f"{language}-{voice_name}-{quality}"
export_voice_path = "/content/project/voice-"+export_voice_name
packages_path = "/content/project/packages"
if not os.path.exists(export_voice_path):
    os.makedirs(export_voice_path)
if not os.path.exists(packages_path):
    os.makedirs(packages_path)
print("Downloading model and his config...")
!gdown -q "{model_id}" -O /content/project/model.ckpt
!gdown -q "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json"
#@markdown **Do you want to write a model card?**
write_model_card = True #@param {type:"boolean"}
if write_model_card:
    with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r") as file:
        config = json.load(file)
    sample_rate = config["audio"]["sample_rate"]
    num_speakers = config["num_speakers"]
    output.eval_js('new Audio("https://github.com/rmcpantoja/piper/raw/master/notebooks/waiting.wav?raw=true").play()')
    text_area = widgets.Textarea(
        description = "fill in this following template and press start to generate the voice package",
        value=f'# Model card for {voice_name} ({quality})\n\n* Language: {language} (normaliced)\n* Speakers: {num_speakers}\n* Quality: {quality}\n* Samplerate: {sample_rate}Hz\n\n## Dataset\n\n* URL: \n* License: \n\n## Training\n\nTrained from scratch.\nOr finetuned from: ',
        layout=widgets.Layout(width='500px', height='200px')
    )
    button = widgets.Button(description='Start')

    def create_model_card(button):
        model_card_text = text_area.value.strip()
        with open(f'{export_voice_path}/MODEL_CARD', 'w') as file:
            file.write(model_card_text)
        text_area.close()
        button.close()
        output.clear()
        output.eval_js('new Audio("https://github.com/rmcpantoja/piper/blob/master/notebooks/starting.wav?raw=true").play()')
        start_process()

    button.on_click(create_model_card)

    display(text_area, button)

def start_process():
    !python -m piper_train.export_onnx "/content/project/model.ckpt" "{export_voice_path}/{export_voice_name}.onnx"
    print("compressing...")
    !tar -czvf "{packages_path}/voice-{export_voice_name}.tar.gz" -C "{export_voice_path}" .
    output.eval_js('new Audio("https://github.com/rmcpantoja/piper/raw/master/notebooks/success.wav?raw=true").play()')

In [ ]:
#@title Download generated voice package
from google.colab import files
files.download(f"{packages_path}/voice-{export_voice_name}.tar.gz")